# 安装 unsloth

In [1]:
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

Looking in indexes: https://download.pytorch.org/whl/cu128
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.0/901.0 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


# 日志设置

In [3]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import sys
import logging

program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)

# 指令设计

In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction: 
You are an expert of film comment analysis. Analyze the given text from an online review and determine the sentiment polarity. Respond with 'positive' or 'negative'.   

### Question: 
{} 

### Response: 
{}
"""

In [5]:
# 提示模板 进行微调需要补充全部的推理指令
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction: 
You are an expert of film comment analysis. Analyze the given text from an online review and determine the sentiment polarity. Respond with 'positive' or 'negative'.   

### Question: 
{} 

### Response: 
"""

# 加载模型

In [6]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=wandb_api_key)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

True

In [7]:
from huggingface_hub import login 
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient() 
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN") 
login(hf_token) 

In [8]:
from unsloth import FastLanguageModel 

logger.info("Loading Qwen model...")

model_name = "unsloth/Qwen3-0.6B-unsloth-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained( 
    model_name = model_name, 
    max_seq_length = 2048, 
    dtype = None, 
    load_in_4bit = True, 
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-15 05:50:02.680755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763185802.847291      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763185802.895546      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-15 05:50:14,075: INFO: NumExpr defaulting to 4 threads.
2025-11-15 05:50:21,808: INFO: TensorFlow version 2.18.0 available.
2025-11-15 05:50:21,810: INFO: JAX version 0.5.2 available.


🦥 Unsloth Zoo will now patch everything to make training faster!


2025-11-15 05:50:32,683: INFO: Loading Qwen model...


==((====))==  Unsloth 2025.11.2: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33+5d4b92a5.d20251029. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

# 加载数据

In [9]:
EOS_TOKEN = tokenizer.eos_token
 
def formatting_prompts_func(examples): 
    inputs = examples["text"] 
    labels = examples["label"]
    outputs = []

    for input, label in zip(inputs, labels): 
        label_text = "positive" if label == 1 else "negative"
        text = alpaca_prompt.format(input, label_text) + EOS_TOKEN 
        outputs.append(text) 
    return { 
        "text": outputs, 
    } 

In [10]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split

train = pd.read_csv("/kaggle/input/corpus-imdb/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("/kaggle/input/corpus-imdb/testData.tsv", header=0, delimiter="\t", quoting=3)

train, val = train_test_split(train, test_size=.2)

train_dict = {'label': train["sentiment"], 'text': train['review']}
val_dict = {'label': val["sentiment"], 'text': val['review']}
test_dict = {"text": test['review']}

train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)
test_dataset = Dataset.from_dict(test_dict)

In [11]:
# 数据格式化
logger.info("Formatting datasets...")
train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
val_dataset = val_dataset.map(formatting_prompts_func, batched = True,)

2025-11-15 05:50:59,826: INFO: Formatting datasets...


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

# LoRA 微调

In [12]:
logger.info("Setting up PEFT...")
model = FastLanguageModel.get_peft_model( 
    model, 
    r=16,
    lora_alpha=16, 
    lora_dropout=0,   
    bias="none",
    target_modules=[ 
        "q_proj", 
        "k_proj", 
        "v_proj", 
        "o_proj", 
        "gate_proj", 
        "up_proj", 
        "down_proj", 
    ], 

    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
 
    random_state=3407, 
    use_rslora=False,   
    loftq_config=None, 
) 

logger.info("Model parameters:" + str(sum(p.numel() for p in model.parameters())))

2025-11-15 05:51:00,344: INFO: Setting up PEFT...
Unsloth 2025.11.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.
2025-11-15 05:51:06,591: INFO: Model parameters:398524416


# SFT 监督微调

In [13]:
import torch
from transformers import TrainingArguments 

# 训练参数
os.makedirs('/kaggle/working/checkpoint', exist_ok=True)

logger.info("Setting up Training Arguments...")
training_args = TrainingArguments(
    output_dir='/kaggle/working/checkpoint',
    per_device_train_batch_size=16,
    # gradient_accumulation_steps=4,
    warmup_steps=100,
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    save_strategy="epoch",
    eval_strategy="epoch",
)

2025-11-15 05:51:06,867: INFO: Setting up Training Arguments...


In [14]:
from trl import SFTTrainer 
from unsloth import is_bfloat16_supported 

logger.info("Initializing SFTTrainer...")
trainer = SFTTrainer( 
    model=model, 
    tokenizer=tokenizer, 
    train_dataset=train_dataset, 
    eval_dataset=val_dataset,
    dataset_text_field="text", 
    max_seq_length=2048, 
    dataset_num_proc=1,
    packing=False, 
    args=training_args,
)

2025-11-15 05:51:07,035: INFO: Initializing SFTTrainer...


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/20000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/5000 [00:00<?, ? examples/s]

# 训练

In [15]:
logger.info("Starting training...")
trainer_stats = trainer.train() 
print(trainer_stats)

2025-11-15 05:51:29,029: INFO: Starting training...
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 3 | Total steps = 3,750
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 10,092,544 of 606,142,464 (1.67% trained)
wandb: Tracking run with wandb version 0.21.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20251115_055131-jqibtrzy
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run peachy-haze-20
wandb: ⭐️ View project at https://wandb.ai/ningjingke-yunnan-university/huggingface
wandb: 🚀 View run at https://wandb.ai/ningjingke-yunnan-university/huggingface/runs/jqibtrzy


Epoch,Training Loss,Validation Loss
1,2.631100,2.563667
2,2.552100,2.542652
3,2.578800,2.537685


Unsloth: Will smartly offload gradients to save VRAM!
TrainOutput(global_step=3750, training_loss=2.591763659667969, metrics={'train_runtime': 18857.6697, 'train_samples_per_second': 3.182, 'train_steps_per_second': 0.199, 'total_flos': 1.4060058574848e+17, 'train_loss': 2.591763659667969, 'epoch': 3.0})


# 推理

In [16]:
logger.info("Starting inference...")
FastLanguageModel.for_inference(model)

test_texts = test_dataset['text']
test_ids = test['id']
predictions = []


for review_text in test_texts:
    prompt = prompt_style.format(review_text)
    inputs = tokenizer([prompt], return_tensors="pt", truncation=True, max_length=2048).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=3, eos_token_id=tokenizer.eos_token_id)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    try:
        response_part = generated_text.split("### Response:")[1].strip().lower()

        if "positive" in response_part:
            predictions.append(1)
        elif "negative" in response_part:
            predictions.append(0)
        else:
            predictions.append(0)

    except (IndexError, AttributeError):
        logger.warning(f"Failed to parse model output: {generated_text}")
        predictions.append(0)

2025-11-15 11:05:49,308: INFO: Starting inference...
2025-11-15 11:08:39,577: WARNING: Failed to parse model output: Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction: 
You are an expert of film comment analysis. Analyze the given text from an online review and determine the sentiment polarity. Respond with 'positive' or 'negative'.   

### Question: 
"There's a sign on The Lost Highway that says:<br /><br />*MAJOR SPOILERS AHEAD*<br /><br />(but you already knew that, didn't you?)<br /><br />Since there's a great deal of people that apparently did not get the point of this movie, I'd like to contribute my interpretation of why the plot makes perfect sense. As others have pointed out, one single viewing of this movie is not s

In [17]:
logger.info("Saving results...")
result_output = pd.DataFrame(data={"id": test_ids, "sentiment": predictions})
result_output.to_csv("/kaggle/working/qwen3_4b_instruct_unsloth.csv", index=False, quoting=3)
logger.info('Qwen result saved!')

2025-11-15 13:02:43,521: INFO: Saving results...
2025-11-15 13:02:43,553: INFO: Qwen result saved!
